In [1]:
## import the tools
import torch
import glob
from pathlib import Path
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN
import glob
import pandas as pd
import ipdb
import laspy
import open3d as o3d
from tqdm.notebook import tqdm
import numpy as np


import open3d as o3d
import laspy


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
### TEST DATA FILENAMES 
#VAL FILES
filename = "PUNKTSKY_00005_1km_6220_495" # OKAY EVEN THO IT IS SRAIGHT LINE
filename = "PUNKTSKY_00005_1km_6221_452" # REALLY BAD EVEN THO IT IS SRAIGHT LINE
# TEST FILES
filename = "PUNKTSKY_00005_1km_6219_494" 
filename = "PUNKTSKY_00005_1km_6211_474" 


In [3]:
# eval_file = "/home/jfriis/msc_data/eval/"+filename+".laz"
# data_val_path_file = "/home/jf/data/denmark/raw/val/"+filename+".laz"
path_to_file = "/home/jf/data/denmark/raw/test/"+filename+".laz"
path = "/home/jf/data/denmark/raw/"

## COUNT THE AMOUNT OF POINTS

In [4]:
total_points = 0
total_pl_points = 0
amount_of_files = 0
dirs = list(glob.glob(path+"*"))
for dir in dirs:
    files = list(glob.glob(dir+"/*.l*"))
    for file in files:
        data = laspy.read(file, laz_backend=laspy.compression.LazBackend.LazrsParallel)
        label_data = data[data.classification == 14]
        amount_of_points = len(data)
        amount_of_powerline_points = len(label_data)
        total_points += amount_of_points
        total_pl_points += amount_of_powerline_points
        amount_of_files += 1
        print(file.split('/')[-1].replace(".laz",''))
        print(f"Amount of points {amount_of_points} amount of powerline points {amount_of_powerline_points} percentage of interseting points {amount_of_powerline_points/amount_of_points * 100 :.4f}")

print(f"Average amount of points: {total_points/amount_of_files:.0f}. Average amount of pl points: {total_pl_points/amount_of_files:.0f}. Average percentage of pl points: {total_pl_points/total_points * 100 :.4f}")


4095
PUNKTSKY_00004_1km_6106_492
Amount of points 21352653 amount of powerline points 4415 percentage of interseting points 0.0207
4095
PUNKTSKY_00005_1km_6219_494
Amount of points 19684629 amount of powerline points 18363 percentage of interseting points 0.0933
4095
PUNKTSKY_00004_1km_6106_510
Amount of points 13528830 amount of powerline points 5502 percentage of interseting points 0.0407
4095
PUNKTSKY_00004_1km_6106_494
Amount of points 29299912 amount of powerline points 6631 percentage of interseting points 0.0226
4095
PUNKTSKY_00005_1km_6211_474
Amount of points 15599105 amount of powerline points 31418 percentage of interseting points 0.2014
4095
PUNKTSKY_00004_1km_6105_518
Amount of points 16880079 amount of powerline points 26903 percentage of interseting points 0.1594
4095
PUNKTSKY_00004_1km_6106_493
Amount of points 29798574 amount of powerline points 11865 percentage of interseting points 0.0398
4095
PUNKTSKY_00005_1km_6205_513
Amount of points 21434582 amount of powerline 

### SEE HOW MUCH REMOVING THE BOTTOM 3.5 meters does

In [15]:
total_points = 0
total_remove_points = 0
dirs = list(glob.glob(path+"*"))
mean_removed = []
for dir in dirs:
    files = list(glob.glob(dir+"/*.l*"))
    for file in files:
        whole_data = laspy.read(file, laz_backend=laspy.compression.LazBackend.LazrsParallel)
        filename = file.split('/')[-1].replace(".laz",'')
        no_bottom_data = laspy.read(f"{dir}/LazFilesWithHeightRemoved/{filename+'_height_filtered.laz'}", laz_backend=laspy.compression.LazBackend.LazrsParallel)
        amount_of_points = len(whole_data)
        amount_of_points_no_bottom = len(no_bottom_data)


        print(file.split('/')[-1].replace(".laz",''))
        print(f"Amount of points {amount_of_points} amount of when bottom is removed {amount_of_points_no_bottom} percentage removed {100-(amount_of_points_no_bottom/amount_of_points * 100) :.4f}")
        mean_removed.append(100-(amount_of_points_no_bottom/amount_of_points * 100))

print(np.mean(mean_removed))

4095
4095
PUNKTSKY_00004_1km_6106_492
Amount of points 21352653 amount of when bottom is removed 1069349 percentage removed 94.9920
4095
4095
PUNKTSKY_00005_1km_6219_494
Amount of points 19684629 amount of when bottom is removed 3227354 percentage removed 83.6047
4095
4095
PUNKTSKY_00004_1km_6106_510
Amount of points 13528830 amount of when bottom is removed 828094 percentage removed 93.8790
4095
4095
PUNKTSKY_00004_1km_6106_494
Amount of points 29299912 amount of when bottom is removed 5844372 percentage removed 80.0533
4095
4095
PUNKTSKY_00005_1km_6211_474
Amount of points 15599105 amount of when bottom is removed 3029181 percentage removed 80.5811
4095
4095
PUNKTSKY_00004_1km_6105_518
Amount of points 16880079 amount of when bottom is removed 3588890 percentage removed 78.7389
4095
4095
PUNKTSKY_00004_1km_6106_493
Amount of points 29798574 amount of when bottom is removed 8716278 percentage removed 70.7493
4095
4095
PUNKTSKY_00005_1km_6205_513
Amount of points 21434582 amount of whe

ZeroDivisionError: division by zero

# HOUGH DATA

In [16]:
total_points = 0
total_remove_points = 0
dirs = list(glob.glob(path+"*"))
amount_of_files = 0
for dir in dirs:
    files = list(glob.glob(dir+"/*.l*"))
    for file in files:
        try:
            filename = file.split('/')[-1].replace(".laz",'')
            cnn_data = laspy.read(f"{dir}/NewLaz/{filename+'.laz'}", laz_backend=laspy.compression.LazBackend.LazrsParallel)
            amount_of_points = len(cnn_data)
            amount_of_files += 1
            total_points += amount_of_points
            print(file.split('/')[-1].replace(".laz",''))
            print(f"Amount of points {amount_of_points}")
        except:
            pass
        
print(f"Average amount of points: {total_points/amount_of_files:.0f}.")


4095
PUNKTSKY_00004_1km_6106_492
Amount of points 2593600
4095
PUNKTSKY_00005_1km_6219_494
Amount of points 4364059
4095
PUNKTSKY_00004_1km_6106_510
Amount of points 2361774
4095
PUNKTSKY_00004_1km_6106_494
Amount of points 4240278
4095
PUNKTSKY_00005_1km_6211_474
Amount of points 389372
4095
PUNKTSKY_00004_1km_6105_518
Amount of points 2851185
4095
PUNKTSKY_00005_1km_6205_513
Amount of points 10824610
4095
PUNKTSKY_00005_1km_6168_458
Amount of points 2017178
4095
PUNKTSKY_00005_1km_6161_465
Amount of points 6963405
4095
PUNKTSKY_00005_1km_6167_473
Amount of points 10196219
4095
PUNKTSKY_00005_1km_6146_468
Amount of points 6001536
4095
PUNKTSKY_00005_1km_6167_474
Amount of points 15498063
4095
PUNKTSKY_00005_1km_6167_475
Amount of points 14373482
4095
PUNKTSKY_00005_1km_6163_472
Amount of points 11266877
4095
PUNKTSKY_00005_1km_6162_472
Amount of points 7389632
4095
PUNKTSKY_00005_1km_6134_518
Amount of points 2300095
4095
PUNKTSKY_00005_1km_6204_505
Amount of points 12914548
4095
PUNK

### CNN DATA

In [17]:
total_points = 0
total_remove_points = 0
dirs = list(glob.glob(path+"*"))
amount_of_files = 0
for dir in dirs:
    files = list(glob.glob(dir+"/*.l*"))
    for file in files:
        try:
            filename = file.split('/')[-1].replace(".laz",'')
            cnn_data = laspy.read(f"{dir}/CNNLaz/{filename+'.laz'}", laz_backend=laspy.compression.LazBackend.LazrsParallel)
            amount_of_points = len(cnn_data)
            amount_of_files += 1
            total_points += amount_of_points
            print(file.split('/')[-1].replace(".laz",''))
            print(f"Amount of points {amount_of_points}")
        except:
            pass
        
print(f"Average amount of points: {total_points/amount_of_files:.0f}.")


4095
PUNKTSKY_00004_1km_6106_492
Amount of points 176976
4095
PUNKTSKY_00005_1km_6219_494
Amount of points 869770
4095
PUNKTSKY_00004_1km_6106_510
Amount of points 237278
4095
PUNKTSKY_00004_1km_6106_494
Amount of points 429541
4095
PUNKTSKY_00005_1km_6211_474
Amount of points 919663
4095
PUNKTSKY_00004_1km_6105_518
Amount of points 388294
4095
PUNKTSKY_00004_1km_6106_493
Amount of points 640938
4095
PUNKTSKY_00005_1km_6205_513
Amount of points 927727
4095
PUNKTSKY_00005_1km_6168_458
Amount of points 348913
4095
PUNKTSKY_00005_1km_6161_465
Amount of points 391140
4095
PUNKTSKY_00005_1km_6167_473
Amount of points 2244726
4095
PUNKTSKY_00005_1km_6146_468
Amount of points 2059381
4095
PUNKTSKY_00005_1km_6167_474
Amount of points 1390324
4095
PUNKTSKY_00005_1km_6167_475
Amount of points 1764250
4095
PUNKTSKY_00005_1km_6163_472
Amount of points 1705311
4095
PUNKTSKY_00005_1km_6162_472
Amount of points 1879861
4095
PUNKTSKY_00005_1km_6134_518
Amount of points 366188
4095
PUNKTSKY_00005_1km_6

In [5]:
data = laspy.read(path_to_file, laz_backend=laspy.compression.LazBackend.LazrsParallel)
all_point_data = np.stack([data.X, data.Y, data.Z], axis=0).transpose((1, 0))
print(all_point_data.shape)

4095
(15599105, 3)


In [6]:
def voxelization(point_cloud, voxel_size):
    # Calculate the minimum and maximum coordinates of the point cloud
    min_coord = np.min(point_cloud, axis=0)
    max_coord = np.max(point_cloud, axis=0)

    # Calculate the number of voxels in each dimension
    voxel_counts = ((max_coord - min_coord) / voxel_size).astype(int) + 1

    # Create an empty voxel grid
    voxel_grid = np.zeros(voxel_counts, dtype=int)

    # Assign points to their respective voxels
    voxel_indices = ((point_cloud - min_coord) / voxel_size).astype(int)
    voxel_grid[tuple(voxel_indices.T)] = 1

    return voxel_grid


In [7]:
def count_voxels(voxel_grid):
    num_empty_voxels = np.count_nonzero(voxel_grid == 0)
    num_occupied_voxels = np.count_nonzero(voxel_grid == 1)
    
    return num_empty_voxels, num_occupied_voxels


In [8]:
def calculate_sparsity(voxel_grid):
    num_empty_voxels, num_occupied_voxels = count_voxels(voxel_grid)
    total_voxels = voxel_grid.size

    sparsity = num_empty_voxels / total_voxels

    return sparsity


In [9]:
def calculate_point_cloud_density(point_cloud):
    # Count the number of points
    num_points = len(point_cloud)

    # Calculate the volume (e.g., using the bounding box)
    min_coords = np.min(point_cloud, axis=0)
    max_coords = np.max(point_cloud, axis=0)
    volume = np.prod(max_coords - min_coords)

    # Calculate the density
    density = num_points / volume

    return density

In [10]:
# Assuming you have a point cloud represented as a numpy array
point_cloud = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])

# Calculate the density
density = calculate_point_cloud_density(all_point_data)
print("Density:", density)


Density: 1.0815437148998128e-07


In [11]:
# Assuming you have a point cloud represented as a numpy array
point_cloud = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])

# Voxelization
voxel_size = 0.1
voxel_grid = voxelization(all_point_data, voxel_size)

# Calculate sparsity
sparsity = calculate_sparsity(voxel_grid)
print("Sparsity:", sparsity)


MemoryError: Unable to allocate 1.00 EiB for an array with shape (1000001, 1000001, 144231) and data type int64

In [ ]:
def calculate_multiplications_3d_unet():
    # Contracting Path (Encoder)
    contracting_layers = [
        (128, 128, 128, 64),  # Layer 1
        (64, 64, 64, 128),    # Layer 2
        (32, 32, 32, 256),    # Layer 3
        (16, 16, 16, 512)     # Layer 4
    ]

    contracting_multiplications = 0
    for layer in contracting_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 3) * (layer[0] * layer[1] * layer[2] * layer[3])
        contracting_multiplications += layer_multiplications

    # Expanding Path (Decoder)
    expanding_layers = [
        (32, 32, 32, 256),    # Layer 4
        (64, 64, 64, 128),    # Layer 3
        (128, 128, 128, 64),  # Layer 2
        (256, 256, 256, 64)   # Layer 1
    ]

    expanding_multiplications = 0
    for layer in expanding_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 3) * (layer[0] * layer[1] * layer[2] * layer[3])
        expanding_multiplications += layer_multiplications

    # Total number of multiplications
    total_multiplications = contracting_multiplications + expanding_multiplications

    return total_multiplications

# Calculate the total number of multiplications
total_multiplications = calculate_multiplications_3d_unet()
print("Total number of multiplications in the U-Net architecture with 3D input:", total_multiplications)


Total number of multiplications in the U-Net architecture with 3D input: 38560333824


In [ ]:
def calculate_multiplications_2d_unet():
    # Contracting Path (Encoder)
    contracting_layers = [
        (128, 128, 64),  # Layer 1
        (64, 64, 128),    # Layer 2
        (32, 32, 256),    # Layer 3
        (16, 16, 512)     # Layer 4
    ]

    contracting_multiplications = 0
    for layer in contracting_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 2) * (layer[0] * layer[1] * layer[2])
        contracting_multiplications += layer_multiplications

    # Expanding Path (Decoder)
    expanding_layers = [
        (32, 32, 256),    # Layer 4
        (64, 64, 128),    # Layer 3
        (128, 128, 64),  # Layer 2
        (256, 256, 64)   # Layer 1
    ]

    expanding_multiplications = 0
    for layer in expanding_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 2) * (layer[0] * layer[1] * layer[2])
        expanding_multiplications += layer_multiplications

    # Total number of multiplications
    total_multiplications = contracting_multiplications + expanding_multiplications

    return total_multiplications

# Calculate the total number of multiplications
total_multiplications = calculate_multiplications_2d_unet()
print("Total number of multiplications in the U-Net architecture with 2D input:", total_multiplications)


Total number of multiplications in the U-Net architecture with 2D input: 71958528
